In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [30]:
import os
import numpy as np
from tqdm import tqdm, tqdm_notebook

import cv2
from skimage import io
from skimage.color import rgb2gray
from skimage import exposure as ep

from skimage import feature as ft
from sklearn import metrics

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR


import sklearn.svm as svm
import sklearn.neighbors as knn

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
train_dir = 'data/train/'
validation_dir = 'data/validation/'
test_dir = 'data/test/'


output_classes = 2
batch_size = 32 
img_height, img_width = 256, 256

# nb_train_samples = 1188
# nb_validation_samples = 144
# nb_test_samples = 144

In [4]:
def get_data(folder):
    """
    Load the data and labels from the given folder.
    """
    X = []
    y = []
    for folderName in os.listdir(folder):
        if not folderName.startswith('.'):
            if folderName in ['normal']:
                label = 0
            elif folderName in ['malignant']:
                label = 1
#             else:
#                 label = 3
            for image_filename in tqdm_notebook(os.listdir(folder + folderName)):
                img_file = cv2.imread(folder + folderName + '/' + image_filename)
                if img_file is not None:
#                     img_file = skimage.transform.resize(img_file, (imageSize, imageSize, 3))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y

In [5]:
X_train, y_train = get_data(train_dir)

In [6]:
X_validation, y_validation = get_data(validation_dir)

In [7]:
X_test, y_test = get_data(test_dir)

In [8]:
from skimage import feature
import numpy as np

class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
        # store the number of points and radius
        self.numPoints = numPoints
        self.radius = radius

    def describe(self, image, eps=1e-7):
        # compute the Local Binary Pattern representation
        # of the image, and then use the LBP representation
        # to build the histogram of patterns
        lbp = feature.local_binary_pattern(image, self.numPoints,
            self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(),
            bins=np.arange(0, self.numPoints + 3),
            range=(0, self.numPoints + 2))

        # normalize the histogram
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)

        # return the histogram of Local Binary Patterns
        return hist

# settings for LBP
radius = 3
n_points = 8 * radius
desc = LocalBinaryPatterns(24, 8)

In [9]:
trainSetList = []
validationSetList = []
testSetList = []

In [10]:
for imgs in X_train:
    gray = cv2.cvtColor(imgs, cv2.COLOR_BGR2GRAY)
    hist = desc.describe(gray)
    trainSetList.append(hist.ravel())
    pass

In [11]:
for imgs in X_validation:
    gray = cv2.cvtColor(imgs, cv2.COLOR_BGR2GRAY)
    hist = desc.describe(gray)
    validationSetList.append(hist.ravel())
    pass

In [12]:
for imgs in X_test:
    gray = cv2.cvtColor(imgs, cv2.COLOR_BGR2GRAY)
    hist = desc.describe(gray)
    testSetList.append(hist.ravel())
    pass

trainSet = np.array(trainSetList, np.float32)
validationSet = np.array(validationSetList, np.float32)
testSet = np.array(testSetList, np.float32)

In [13]:
# from keras.utils.np_utils import to_categorical
# y_trainHot = to_categorical(y_train, num_classes = 2)
# y_valHot = to_categorical(y_validation, num_classes = 2)
# y_testHot = to_categorical(y_test, num_classes = 2)

In [14]:
trainSet.shape

(59168, 26)

In [15]:
validationSet.shape

(18384, 26)

In [16]:
testSet.shape

(15472, 26)

In [17]:
y_train.shape

(59168,)

In [18]:
# y_trainHot.shape

In [19]:
y_test.shape

(15472,)

In [20]:
y_test

array([1, 1, 1, ..., 0, 0, 0])

In [32]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction
kfold = KFold(n_splits=10, random_state=22) # k=10, split the data into 10 equal parts
xyz=[]
accuracy=[]
std=[]
classifiers=['Linear Svm','Radial Svm','Logistic Regression','KNN','Decision Tree','Naive Bayes','Random Forest', 'Gradient Boosting', 'Adaboost', 'XGBoost']
models=[svm.SVC(kernel='linear'),
        svm.SVC(kernel='rbf'),
        LogisticRegression(),
        KNeighborsClassifier(n_neighbors=9),
        DecisionTreeClassifier(),
        GaussianNB(),
        RandomForestClassifier(n_estimators=100),
        GradientBoostingClassifier(n_estimators=500,random_state=0,learning_rate=0.1),
        AdaBoostClassifier(n_estimators=200,random_state=0,learning_rate=0.1),
        xgb.XGBClassifier(n_estimators=900,learning_rate=0.1)]
for i in models:
    model = i
    cv_result = cross_val_score(model, testSet,y_test, cv = kfold ,scoring = "accuracy")
    cv_result=cv_result
    xyz.append(cv_result.mean())
    std.append(cv_result.std())

new_models_dataframe2=pd.DataFrame({'CV Mean':xyz,'Std':std},index=classifiers)       
print(new_models_dataframe2)

                      CV Mean       Std
Linear Svm           0.755024  0.189672
Radial Svm           0.676018  0.447488
Logistic Regression  0.742869  0.234000
KNN                  0.861029  0.106609
Decision Tree        0.889343  0.050448
Naive Bayes          0.711484  0.045830
Random Forest        0.944152  0.061087
Gradient Boosting    0.903753  0.088194
Adaboost             0.823665  0.171799
XGBoost              0.928316  0.063509


In [72]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction
kfold = KFold(n_splits=10, random_state=22) # k=10, split the data into 10 equal parts
xyz=[]
accuracy=[]
std=[]

model = RandomForestClassifier(n_estimators=100)

cv_result = cross_val_score(model, testSet,y_test, cv = kfold ,scoring = "accuracy")
cv_result=cv_result
xyz.append(cv_result.mean())
std.append(cv_result.std())
accuracy.append(cv_result)
new_models_dataframe2=pd.DataFrame({'CV Mean':xyz,'Std':std})       
print(new_models_dataframe2)
print("folds accuracies:" ,cv_result )

    CV Mean       Std
0  0.943699  0.061443
folds accuracies: [0.9870801  0.9870801  0.99030381 0.98901099 0.98319328 0.95151907
 0.98642534 0.81577246 0.87782805 0.86877828]


In [75]:
df_rf = pd.DataFrame(accuracy, columns=["f1", "f2", "f3", "f4", "f5", "f6", "f7", "f8", "f9", "f10"] )
df_rf

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10
0,0.98708,0.98708,0.990304,0.989011,0.983193,0.951519,0.986425,0.815772,0.877828,0.868778


In [77]:
xyz=[]
accuracy=[]
std=[]

model = xgb.XGBClassifier(n_estimators=900,learning_rate=0.1)

cv_result = cross_val_score(model, testSet,y_test, cv = kfold ,scoring = "accuracy")
cv_result=cv_result
xyz.append(cv_result.mean())
std.append(cv_result.std())
accuracy.append(cv_result)
new_models_dataframe2=pd.DataFrame({'CV Mean':xyz,'Std':std})       
print(new_models_dataframe2)
print("folds accuracies:" ,cv_result )

    CV Mean       Std
0  0.928316  0.063509
folds accuracies: [0.97222222 0.97739018 0.97672915 0.98125404 0.97155785 0.92889463
 0.96767938 0.80155139 0.85261797 0.85326438]


In [70]:
xyz=[]
accuracy=[]
std=[]

model = AdaBoostClassifier(n_estimators=200,random_state=0,learning_rate=0.1)

cv_result = cross_val_score(model, testSet,y_test, cv = kfold ,scoring = "accuracy")
cv_result=cv_result
xyz.append(cv_result.mean())
std.append(cv_result.std())
accuracy.append(cv_result)
new_models_dataframe2=pd.DataFrame({'CV Mean':xyz,'Std':std})       
print(new_models_dataframe2)
print("folds accuracies:" ,cv_result)

    CV Mean       Std
0  0.823665  0.171799
folds accuracies: [0.94379845 0.95801034 0.95281189 0.94828701 0.95087266 0.92049127
 0.86942469 0.53975436 0.56884292 0.58435682]


In [37]:
xyz=[]
accuracy=[]
std=[]

model = GradientBoostingClassifier(n_estimators=500,random_state=0,learning_rate=0.1)

cv_result = cross_val_score(model, testSet,y_test, cv = kfold ,scoring = "accuracy")
cv_result=cv_result
xyz.append(cv_result.mean())
std.append(cv_result.std())
accuracy.append(cv_result)
new_models_dataframe2=pd.DataFrame({'CV Mean':xyz,'Std':std})       
print(new_models_dataframe2)
print("folds accuracies:" ,cv_result)

    CV Mean       Std
0  0.903753  0.088194
folds accuracies: [0.9625323  0.97222222 0.97155785 0.97737557 0.96574014 0.9217841
 0.95022624 0.74337427 0.78409825 0.78862314]


In [38]:
xyz=[]
accuracy=[]
std=[]

model = GaussianNB()

cv_result = cross_val_score(model, testSet,y_test, cv = kfold ,scoring = "accuracy")
cv_result=cv_result
xyz.append(cv_result.mean())
std.append(cv_result.std())
accuracy.append(cv_result)
new_models_dataframe2=pd.DataFrame({'CV Mean':xyz,'Std':std})       
print(new_models_dataframe2)
print("folds accuracies:" ,cv_result)

    CV Mean      Std
0  0.711484  0.04583
folds accuracies: [0.64728682 0.69444444 0.68972204 0.64835165 0.73303167 0.68261151
 0.70523594 0.77052359 0.76793794 0.77569489]


In [39]:
xyz=[]
accuracy=[]
std=[]

model = DecisionTreeClassifier()

cv_result = cross_val_score(model, testSet,y_test, cv = kfold ,scoring = "accuracy")
cv_result=cv_result
xyz.append(cv_result.mean())
std.append(cv_result.std())
accuracy.append(cv_result)
new_models_dataframe2=pd.DataFrame({'CV Mean':xyz,'Std':std})       
print(new_models_dataframe2)
print("folds accuracies:" ,cv_result)

   CV Mean       Std
0  0.89348  0.050467
folds accuracies: [0.93475452 0.92958656 0.9321267  0.93665158 0.91984486 0.87201034
 0.94053006 0.79896574 0.83581125 0.83451842]


In [40]:
xyz=[]
accuracy=[]
std=[]

model = KNeighborsClassifier(n_neighbors=9)

cv_result = cross_val_score(model, testSet,y_test, cv = kfold ,scoring = "accuracy")
cv_result=cv_result
xyz.append(cv_result.mean())
std.append(cv_result.std())
accuracy.append(cv_result)
new_models_dataframe2=pd.DataFrame({'CV Mean':xyz,'Std':std})       
print(new_models_dataframe2)
print("folds accuracies:" ,cv_result)

    CV Mean       Std
0  0.861029  0.106609
folds accuracies: [0.94509044 0.94056848 0.94376212 0.93018746 0.93471235 0.8752424
 0.93341952 0.66386555 0.72786037 0.71557854]


In [41]:
xyz=[]
accuracy=[]
std=[]

model = LogisticRegression()

cv_result = cross_val_score(model, testSet,y_test, cv = kfold ,scoring = "accuracy")
cv_result=cv_result
xyz.append(cv_result.mean())
std.append(cv_result.std())
accuracy.append(cv_result)
new_models_dataframe2=pd.DataFrame({'CV Mean':xyz,'Std':std})       
print(new_models_dataframe2)
print("folds accuracies:" ,cv_result)

    CV Mean    Std
0  0.742869  0.234
folds accuracies: [0.8875969  0.92635659 0.9114415  0.89915966 0.92954105 0.86037492
 0.85197156 0.37233355 0.38590821 0.40400776]


In [42]:
xyz=[]
accuracy=[]
std=[]

model =  svm.SVC(kernel='rbf')

cv_result = cross_val_score(model, testSet,y_test, cv = kfold ,scoring = "accuracy")
cv_result=cv_result
xyz.append(cv_result.mean())
std.append(cv_result.std())
accuracy.append(cv_result)
new_models_dataframe2=pd.DataFrame({'CV Mean':xyz,'Std':std})       
print(new_models_dataframe2)
print("folds accuracies:" ,cv_result)

    CV Mean       Std
0  0.676018  0.447488
folds accuracies: [0.99870801 0.99870801 0.99806076 0.99870718 0.99870718 0.99547511
 0.77181642 0.         0.         0.        ]


In [43]:
xyz=[]
accuracy=[]
std=[]

model =  svm.SVC(kernel='linear')

cv_result = cross_val_score(model, testSet,y_test, cv = kfold ,scoring = "accuracy")
cv_result=cv_result
xyz.append(cv_result.mean())
std.append(cv_result.std())
accuracy.append(cv_result)
new_models_dataframe2=pd.DataFrame({'CV Mean':xyz,'Std':std})       
print(new_models_dataframe2)
print("folds accuracies:" ,cv_result)

    CV Mean       Std
0  0.755024  0.189672
folds accuracies: [0.86369509 0.91085271 0.89269554 0.88106012 0.91531997 0.83968972
 0.84421461 0.45895281 0.46994182 0.4738203 ]


In [56]:
output

,SVM-output
0,0.863695
1,0.910853
2,0.892696
3,0.881060
4,0.915320
5,0.839690
6,0.844215
7,0.458953
8,0.469942
9,0.473820


In [57]:
output.to_csv("folds.csv", index= False, header=True)